In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('./dataset/train_feature_selected.csv')
df_train['datetime_iso'] = pd.to_datetime(df_train['datetime_iso'])

df_test = pd.read_csv('./dataset/test_feature_selected.csv')
df_test['datetime_iso'] = pd.to_datetime(df_test['datetime_iso'])
df_test_date = df_test['datetime_iso']

In [4]:
df_train.head()

,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,rain_1h,clouds,month,yearly_mean,monthly_mean,weekly_mean,hourly_yearly_mean,datetime_iso
0,24.75,23.89,25.76,24.28,25.22,1012.0,95.0,0.82,320.0,0.00,100.0,1,0.492286,0.442561,0.485107,0.061282,1979-01-01 00:00:00+00:00
1,24.58,23.73,25.57,23.99,25.26,1012.0,95.0,0.96,338.0,0.00,100.0,1,0.492286,0.442561,0.485107,0.042564,1979-01-01 01:00:00+00:00
2,26.60,24.06,26.60,26.10,27.39,1012.0,86.0,1.22,339.0,0.00,99.0,1,0.492286,0.442561,0.485107,0.091538,1979-01-01 02:00:00+00:00
3,27.31,24.37,30.90,26.59,28.36,1012.0,84.0,1.08,342.0,0.13,94.0,1,0.492286,0.442561,0.485107,0.257179,1979-01-01 03:00:00+00:00
4,27.41,25.05,31.54,26.58,28.31,1011.0,87.0,0.86,336.0,0.34,100.0,1,0.492286,0.442561,0.485107,0.718974,1979-01-01 04:00:00+00:00


## Modelling using Pycaret:Catboost

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   temp                49368 non-null  float64            
 1   d_point             49368 non-null  float64            
 2   feels               49368 non-null  float64            
 3   min_temp            49368 non-null  float64            
 4   max_temp            49368 non-null  float64            
 5   prssr               49368 non-null  float64            
 6   hum                 49368 non-null  float64            
 7   wind_spd            49368 non-null  float64            
 8   wind_deg            49368 non-null  float64            
 9   rain_1h             0 non-null      float64            
 10  clouds              49368 non-null  float64            
 11  month               49368 non-null  int64              
 12  yearly_mean         49368 non-nu

In [6]:
df_train = df_train.drop(['datetime_iso'],axis = 1)
df_test = df_test.drop(['datetime_iso','rain_1h'],axis = 1)

In [13]:
from pycaret.regression import *
s = setup(
    df_train, 
    target = 'rain_1h',
    normalize = False,
    normalize_method = 'minmax',
    #remove_multicollinearity = True,
    
    session_id = 41)

,Description,Value
0,Session id,41
1,Target,rain_1h
2,Target type,Regression
3,Original data shape,"(341880, 16)"
4,Transformed data shape,"(341880, 16)"
5,Transformed train set shape,"(239315, 16)"
6,Transformed test set shape,"(102565, 16)"
7,Numeric features,15
8,Preprocess,True
9,Imputation type,simple


In [14]:
#best = compare_models()

In [15]:
catboost = create_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.3992,0.6473,0.8045,0.2624,0.3354,1.1879
1,0.3913,0.6090,0.7804,0.2545,0.3318,1.2301
2,0.3978,0.6190,0.7868,0.2636,0.3353,1.2113
3,0.4074,0.6605,0.8127,0.2716,0.3391,1.2184
4,0.4143,0.6873,0.8290,0.2606,0.3423,1.2418
5,0.4023,0.6362,0.7976,0.2517,0.3382,1.2610
6,0.4134,0.6670,0.8167,0.2652,0.3425,1.2120
7,0.4020,0.6409,0.8005,0.2713,0.3360,1.2204
8,0.4094,0.6579,0.8111,0.2488,0.3412,1.2357


In [16]:
predictions = predict_model(catboost, data=df_test)
predictions.head()

,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,clouds,month,yearly_mean,monthly_mean,weekly_mean,hourly_yearly_mean,prediction_label
0,26.59,23.660000,26.590000,26.020000,27.160000,1009.0,84.0,1.45,355.0,97.0,1,0.492286,0.442561,0.485107,0.061282,-0.126130
1,26.51,24.920000,26.510000,26.059999,28.040001,1009.0,91.0,1.67,351.0,95.0,1,0.492286,0.442561,0.485107,0.042564,-0.019596
2,28.68,25.709999,34.680000,28.030001,29.299999,1009.0,84.0,1.72,345.0,90.0,1,0.492286,0.442561,0.485107,0.091538,0.029955
3,28.84,25.250000,34.509998,28.520000,29.080000,1008.0,81.0,1.49,339.0,91.0,1,0.492286,0.442561,0.485107,0.257179,0.038122
4,29.75,24.620001,35.380001,29.309999,30.570000,1007.0,74.0,1.39,339.0,96.0,1,0.492286,0.442561,0.485107,0.718974,0.214519


In [17]:
submission = pd.DataFrame()
submission['datetime_iso'] = df_test_date
submission['rain_1h'] = predictions['prediction_label']
submission.head()

,datetime_iso,rain_1h
0,2018-01-01 00:00:00+00:00,-0.126130
1,2018-01-01 01:00:00+00:00,-0.019596
2,2018-01-01 02:00:00+00:00,0.029955
3,2018-01-01 03:00:00+00:00,0.038122
4,2018-01-01 04:00:00+00:00,0.214519


In [18]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   datetime_iso  49368 non-null  datetime64[ns, UTC]
 1   rain_1h       49368 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 771.5 KB


In [19]:
submission.to_csv('./submission/submission_9.csv', index=False)